In [216]:
import pandas as pd
from autogluon.tabular import TabularPredictor
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

In [217]:
import ray

# Ray가 실행 중인지 확인 후 종료
if ray.is_initialized():
    print("Ray 셧다운 중...")
    ray.shutdown()
    print("Ray가 성공적으로 종료되었습니다.")
else:
    print("Ray가 실행 중이 아닙니다.")

Ray 셧다운 중...
Ray가 성공적으로 종료되었습니다.


In [218]:
import pandas as pd
from autogluon.tabular import TabularPredictor

# ✅ 데이터 로드
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

In [219]:
train["총 생성 배아 수"] = pd.to_numeric(train["총 생성 배아 수"], errors="coerce")
test["총 생성 배아 수"] = pd.to_numeric(test["총 생성 배아 수"], errors="coerce")
train["저장된 배아 수"] = pd.to_numeric(train["저장된 배아 수"], errors="coerce")
test["저장된 배아 수"] = pd.to_numeric(test["저장된 배아 수"], errors="coerce")

In [220]:
def preprocess_data(df):

    # ✅ 피처 엔지니어링 적용
    df["배아 수 제곱"] = df["이식된 배아 수"] ** 2
    
    df["총 생성 배아 수"] = pd.to_numeric(df["총 생성 배아 수"], errors='coerce')
    df["저장된 배아 수"] = pd.to_numeric(df["저장된 배아 수"], errors='coerce')
    
    df["세부 시술 유형"] = df["세부 시술 유형"].astype(str).str.strip() \
        .str.replace(r"\s*([:/])\s*", r"\1", regex=True) \
        .str.replace(r"\s+", " ", regex=True)

    # ✅ 연령대를 중앙값으로 변환하여 숫자로 변환
    age_mapping = {
        "만18-34세": 26,
        "만35-37세": 36,
        "만38-39세": 38.5,
        "만40-42세": 41,
        "만43-44세": 43.5,
        "만45-50세": 47.5,
        "알 수 없음": np.nan
    }

    # ✅ 매핑 적용
    df["시술 당시 나이 숫자"] = df["환자 시술 당시 나이"].map(age_mapping)

    # 배아 개수 대비 저장된 배아 수 비율 (배아 보존율)
    df["배아 보존율"] = df["저장된 배아 수"] / (df["총 생성 배아 수"] + 1)
    
    # ✅ 나이와 배아 수의 비율 변수 생성
    df["배아 수 대비 나이"] = df["시술 당시 나이 숫자"] / (df["이식된 배아 수"] + 1)
    
    # ✅ 추가 비율 Feature 생성
    df["배아 이식 경과일 대비 나이"] = df["배아 이식 후 경과일"] / (df["시술 당시 나이 숫자"] + 1)
    
    # df["1~2개 이식 여부"] = (df["이식된 배아 수"].between(1, 2)).astype(int)
    
    # df["나이 26~36 여부"] = ((df["시술 당시 나이 숫자"] >= 26) &
    #                     (df["시술 당시 나이 숫자"] <= 36)).astype(int)

    # 1️⃣ 이식된 배아 수 × 시술 당시 나이 숫자
    df["이식된 배아 수 × 시술 당시 나이"] = df["이식된 배아 수"] * df["시술 당시 나이 숫자"]

    # 2️⃣ 배아 이식 경과일 대비 나이 × 배아 수 대비 나이
    df["배아 이식 경과일 대비 나이 × 배아 수 대비 나이"] = df["배아 이식 경과일 대비 나이"] * df["배아 수 대비 나이"]

    # 3️⃣ 배아 수 대비 나이 / 총 생성 배아 수
    #df["배아 수 대비 나이 / 총 생성 배아 수"] = df["배아 수 대비 나이"] / (df["총 생성 배아 수"] + 1)

    # 4️⃣ 이식된 배아 수 × 배아 이식 경과일 대비 나이
    df["이식된 배아 수 × 배아 이식 경과일 대비 나이"] = df["이식된 배아 수"] * df["배아 이식 경과일 대비 나이"]

    # 5️⃣ 시술 당시 나이 숫자 / 총 생성 배아 수
    #df["시술 당시 나이 숫자 / 총 생성 배아 수"] = df["시술 당시 나이 숫자"] / (df["총 생성 배아 수"] + 1)

    drop_columns = [
        "시술 유형"
    ]

    df = df.drop(columns=[col for col in drop_columns if col in df.columns], errors="ignore")

    return df


In [221]:
train = preprocess_data(train)
test = preprocess_data(test)

In [222]:
# ✅ ID 제거
train = train.drop(columns=["ID"])
test_id = test["ID"]
test= test.drop(columns=["ID"])

In [223]:
# ✅ 모델 저장 경로
save_path = "헌주_저장"

In [224]:
train['임신 성공 확률'] = (train['임신 성공 확률'] > 0.8).astype(int)

In [225]:
y_binary = (train["임신 성공 확률"] > 0.5).astype(int)
pos_ratio = y_binary.mean()
neg_ratio = 1 - pos_ratio
scale_pos_weight = round(neg_ratio / pos_ratio, 2)

In [226]:
hyperparameters = {
    'GBM': [{
        'is_unbalance': True
    }],
    'CAT': [{
        #'ag_args_fit': {'num_gpus': 1},
        'class_weights': [1.0, scale_pos_weight]
    }],
    'XGB': [{
        'ag_args_fit': {'num_gpus': 1},
        'tree_method': 'gpu_hist',
        'predictor': 'gpu_predictor',
        'scale_pos_weight': scale_pos_weight
    }],
         'RF': [
        {'n_estimators': 300, 'max_depth': 10, 'min_samples_leaf': 4, 'class_weight': 'balanced_subsample'},

    ],
    
    # ✅ 튜닝 가능한 ExtraTrees
    'XT': [
        {'n_estimators': 500, 'max_depth': 10, 'min_samples_leaf': 4, 'class_weight': {0: 1.0, 1: 3.5}} ]
   # 'KNN': [{}]
}


In [227]:
from sklearn.metrics import make_scorer, f1_score
import numpy as np

In [228]:
from autogluon.core.metrics import make_scorer
from custom_metric import custom_mixed_score  # ✅ import 잘 하셨고

# ⬇️ 여기 추가해주세요!
custom_metric = make_scorer(
    name="mixed_score",
    score_func=custom_mixed_score,
    greater_is_better=True
)

In [229]:
from autogluon.core.metrics import make_scorer
import numpy as np
from sklearn.metrics import f1_score
from autogluon.tabular import TabularPredictor
from autogluon.core.metrics import make_scorer
from custom_metric import custom_mixed_score

# AutoGluon Predictor 설정
predictor = TabularPredictor(
    label="임신 성공 확률",
    problem_type="binary",  # 또는 binary (필요 시)
    eval_metric=custom_metric,
    path=save_path
)

# 학습
predictor.fit(
    train_data=train,
    presets="best_quality",
    time_limit=3600*4,
    num_bag_folds= 8,
    num_stack_levels=0,
    dynamic_stacking=False,
    save_space=True,
    hyperparameters=hyperparameters,
    hyperparameter_tune_kwargs = {
        "num_trials": 20,  # 🚀 각 모델당 20번의 랜덤 탐색
        "scheduler": "local",
        "searcher": "random"
    }
)


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.12.2
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #14~22.04.1-Ubuntu SMP Thu Oct  3 20:59:06 UTC 2024
CPU Count:          128
Memory Avail:       955.79 GB / 1007.51 GB (94.9%)
Disk Space Avail:   106.70 GB / 145.19 GB (73.5%)
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ... Time limit = 14400s
AutoGluon will save models to "/home/ubuntu/workspace/헌주_저장"
Train Data Rows:    126244
Train Data Columns: 40
Label Column:       임신 성공 확률
Problem Type:       binary
Preprocessing data ...
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    978813.39 MB
	Train Data (Original)  Memory Usage: 1

In [230]:
y_pred_proba = predictor.predict_proba(test)
submission = pd.DataFrame({"ID": test_id, "probability": y_pred_proba [1]})

In [231]:
# 결과 병합 및 저장
submission.to_csv("Submission_hj_k15.csv", index=False)

print("ok")

ok


In [232]:
importance = predictor.feature_importance(train)

Computing feature importance via permutation shuffling for 40 features using 5000 rows with 5 shuffle sets...
	451.51s	= Expected runtime (90.3s per shuffle set)
	123.64s	= Actual runtime (Completed 5 of 5 shuffle sets)


In [233]:
importance.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40 entries, 이식된 배아 수 to 총 생성 배아 수
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   importance  40 non-null     float64
 1   stddev      40 non-null     float64
 2   p_value     40 non-null     float64
 3   n           40 non-null     int64  
 4   p99_high    40 non-null     float64
 5   p99_low     40 non-null     float64
dtypes: float64(5), int64(1)
memory usage: 2.2+ KB


In [234]:
importance.head(25)

,importance,stddev,p_value,n,p99_high,p99_low
이식된 배아 수,0.022415,0.005857,0.000512,5,0.034475,0.010354
저장된 배아 수,0.020230,0.002466,0.000026,5,0.025309,0.015152
배아 수 대비 나이,0.011616,0.003891,0.001309,5,0.019628,0.003603
이전 총 임신 횟수,0.010916,0.003549,0.001171,5,0.018224,0.003608
채취된 신선 난자 수,0.010608,0.003250,0.000937,5,0.017299,0.003916
시술 당시 나이 숫자,0.010312,0.004995,0.004954,5,0.020596,0.000028
정자 출처,0.009549,0.003091,0.001152,5,0.015913,0.003185
난자 출처,0.009316,0.005296,0.008526,5,0.020221,-0.001588
배아 이식 경과일 대비 나이,0.008263,0.004355,0.006619,5,0.017231,-0.000705
이전 총 임신 성공 횟수,0.004848,0.000400,0.000005,5,0.005671,0.004025


In [235]:
importance.tail(20)

,importance,stddev,p_value,n,p99_high,p99_low
미세주입(ICSI) 배아 이식 수,0.002427,0.001123,0.004219,5,0.004740,0.000115
이식된 배아 수 × 배아 이식 경과일 대비 나이,0.002391,0.000836,0.001532,5,0.004112,0.000671
불임 원인 - 불명확,0.002223,0.002264,0.046579,5,0.006885,-0.002439
착상 전 PGD 시행 여부,0.001867,0.001704,0.035240,5,0.005376,-0.001642
배아 이식 경과일 대비 나이 × 배아 수 대비 나이,0.001647,0.002791,0.128761,5,0.007393,-0.004100
착상 전 PGS 시행 여부,0.001531,0.001577,0.047884,5,0.004777,-0.001716
배아 이식 후 경과일,0.001502,0.004138,0.231285,5,0.010022,-0.007018
이전 DI 시술 횟수,0.001366,0.002599,0.152501,5,0.006717,-0.003985
배아 수 제곱,0.001332,0.001713,0.078492,5,0.004859,-0.002195
불임 원인 - 난관 질환,0.001236,0.001404,0.060193,5,0.004126,-0.001654
